In [9]:
# Import from local package
import sys
sys.path.append('../model')
import torch
import numpy as np
from tqdm import tqdm as progress_bar

# Device configuration
device = torch.device("cpu")

In [2]:
### import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import FormatStrFormatter
#mpl.use('Agg')
mpl.style.use('classic')

In [3]:
from models import SimpleCNN2Layer as Model
from collectdata import collect_data

In [5]:
_, _, dataset_test = collect_data(
    "/data/schreihf/PvFinder/July_31_75000.npz",
    30000, 10000,
    device=device)

Loading /data/schreihf/PvFinder/July_31_75000.npz
Training: 30000 Validation: 10000 Test: 35000


In [6]:
name = 'July_31_75000_2layer_{}.pyt'
model = Model().to(device)
model.load_state_dict(torch.load(name.format(25)))
model.eval()

SimpleCNN2Layer(
  (conv1): Conv1d(1, 5, kernel_size=(25,), stride=(1,), padding=(12,))
  (conv2): Conv1d(5, 1, kernel_size=(15,), stride=(1,), padding=(7,))
  (fc1): Linear(in_features=4000, out_features=4000, bias=True)
)

In [10]:
# Check False Positive
def testNet2(model, batch_size):
    FP_list=[]
    N=0
    Times=0
    NO_Batch=0
    FP=0
    
    test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size)
    
    with torch.no_grad():
        for inputs, labels in progress_bar(test_loader):
            NO_Batch = NO_Batch +1
            
            #Forward pass
            outputs = model(inputs)
            prob=0
            Tprob=0
            j=0

            while j < 3900:
                for i in range (j,j+1):
                    if outputs[0][i].item()> 1e-2 and labels[0][i].item()==0:
                        for jj in range (i,i+10):
                            Tprob+=labels[0][jj].item()
                            prob+=outputs[0][jj].item()
                            if outputs[0][jj]>0.01:
                                Times=Times+1      
                        if Tprob<0.9 and prob>0.2 and Times>=3:
                            FP=FP+1
                            FP_list.append((NO_Batch, j))
                            j+=10
                        else:
                            j+=1
                    else:
                        j+=1
          
                            
    return FP

In [11]:
FP = testNet2(model, batch_size=1)
print(FP)

100%|██████████| 35000/35000 [07:26<00:00, 78.32it/s]

2834
